In [1]:
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import pandas as pd
import spotipy
import requests
import json
import csv

### Spotify Credentials

In [2]:
user_id = ''
client_id = ''
client_secret = ''

In [3]:
# Spotify API Authorization
spotify_object = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)) 

In [5]:
'''
@ Spotify song details function
'''
def spotify_api(artist_name, search_query):
    
    # spotify api
    searchResults = spotify_object.search(q="artist:" + artist_name + " track:" + search_query, type="track")
    # results
    song_id = searchResults['tracks']['items'][0]['id']  #song id
    uri_id = searchResults['tracks']['items'][0]['uri']  #uri_id
    release_date = searchResults['tracks']['items'][0]['album']['release_date']  #release_date
    type = searchResults['tracks']['items'][0]['type']  #type
    explicit_content = searchResults['tracks']['items'][0]['explicit']  #explicit content
    popularity = searchResults['tracks']['items'][0]['popularity']  #popularity
    
    return [song_id, uri_id, release_date, type, explicit_content, popularity]    

In [7]:
'''
@ Spotify track audio analysis function
'''
def spotify_track_audio_attributes(id):
    song_attributes = spotify_object.audio_features(id)
    danceability = song_attributes[0]['danceability']
    energy = song_attributes[0]['energy']
    loudness = song_attributes[0]['loudness']
    speechiness = song_attributes[0]['speechiness']
    acousticness = song_attributes[0]['acousticness']
    instrumentalness = song_attributes[0]['instrumentalness']
    liveness = song_attributes[0]['liveness']
    valence = song_attributes[0]['valence']
    tempo = song_attributes[0]['tempo']
    
    return [danceability, energy, loudness, speechiness, acousticness, instrumentalness, liveness, valence, tempo]
    

### library and playlist combine data

In [8]:
'''
@ JSON parsing function
'''
def json_extract(data, songs_data):
    for i in data:
        f = open(i, encoding='utf8', errors='ignore')
        # change the JSON string into a JSON object
        jsonObject = json.load(f)
        
        # create a tuple of artist, album, track, uri
        if 'tracks' in jsonObject:
            for key in jsonObject['tracks']:
                artist = key['artist']      # artist name
                album = key['album']        # album name
                track_name = key['track']   # track name
                
                # call to api_data function
                api_data = spotify_api(artist, track_name)  # function
                song_id = api_data[0]       # song_id
                uri_id = api_data[1]        # uri_id
                release_date = api_data[2]  # release_date
                type = api_data[3]          # type
                explicit = api_data[4]      # explicit
                popularity = api_data[5]    # popularity
                
                # call to track_analysis_attributes function
                track_attribute = spotify_track_audio_attributes(song_id)
                danceability = track_attribute[0]
                energy = track_attribute[1]
                loudness = track_attribute[2]
                speechiness = track_attribute[3]
                acousticness = track_attribute[4]
                instrumentalness = track_attribute[5]
                liveness = track_attribute[6]
                valence = track_attribute[7]
                tempo = track_attribute[8]
                
                # adding all the valus in a tuple
                songs_data.append((song_id, track_name, album, uri_id, release_date, explicit, type, popularity, danceability, energy, loudness, speechiness, acousticness, instrumentalness, liveness, valence, tempo))
                    
        elif 'playlists' in jsonObject:
            for key in jsonObject['playlists']:
                for i in key['items']:
                    artist = i['track']['artistName']
                    album = i['track']['albumName']
                    track_name = i['track']['trackName']
                    
                    # call to api_data function
                    api_data = spotify_api(artist, track_name)  # function
                    song_id = api_data[0]       # song_id
                    uri_id = api_data[1]        # uri_id
                    release_date = api_data[2]  # release_date
                    type = api_data[3]          # type
                    explicit = api_data[4]      # explicit
                    popularity = api_data[5]    # popularity

                    # call to track_analysis_attributes function
                    track_attribute = spotify_track_audio_attributes(song_id)
                    danceability = track_attribute[0]       # danceability
                    energy = track_attribute[1]             # energy
                    loudness = track_attribute[2]           # loudness
                    speechiness = track_attribute[3]        # speechiness
                    acousticness = track_attribute[4]       # acousticness
                    instrumentalness = track_attribute[5]   # instrumentalness
                    liveness = track_attribute[6]           # liveness
                    valence = track_attribute[7]            # valence
                    tempo = track_attribute[8]              # tempo
                    
                    songs_data.append((song_id, track_name, album, uri_id, release_date, explicit, type, popularity, danceability, energy, loudness, speechiness, acousticness, instrumentalness, liveness, valence, tempo))
    return songs_data

In [9]:
if __name__ == '__main__':
    data_df = []
    x = json_extract(['./dataset/YourLibrary.json', './dataset/Playlist1.json'], data_df)
    with open('./dataset/songs_data.csv', 'w', newline='', encoding='utf-8') as f:
        header = ['artist_name', 'album_name', 'track_name', 'song_uri']
        writer = csv.writer(f)
        writer.writerow(header)
        writer.writerows(data_df)

IndexError: list index out of range

### Stream history

In [ ]:
stream_history = pd.read_json('./dataset/StreamingHistory0.json')
stream_history.to_csv('./dataset/stream_history.csv', index=False)